In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
r = requests.get('http://www.usclimatedata.com/climate/united-states/us')
print(len(r.text))

42419


In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text)
print(soup.title)
print(soup.title.string) 
# convert our HTML text into a BeautifulSoup object, then get specific tags out of that HTML document

<title>Climate United States - Normals and averages</title>
Climate United States - Normals and averages


In [26]:
# Drill into the bs4 object to access page contents
print(soup.p)
print(soup.p.text)
print(soup.a)
print(soup.a['title'])
print()
print(soup.p.parent)

<p class="lead">Share</p>
Share
<a class="navbar-brand" href="/" title="Temperature - Precipitation - Sunshine - Snowfall"><img alt="Temperature - Precipitation - Sunshine - Snowfall" data-src="https://www.usclimatedata.com/assets/images/us-climate-data.png" height="34" src="https://www.usclimatedata.com/assets/images/us-climate-data.png" srcset="https://www.usclimatedata.com/assets/images/us-climate-data.png 1x, https://www.usclimatedata.com/assets/images/us-climate-data-2.png 2x" width="31"/><span class="white ml-2">U.S. Climate Data</span></a>
Temperature - Precipitation - Sunshine - Snowfall

<div class="box" id="share"><p class="lead">Share</p><a aria-label="" class="resp-sharing-button__link" href=" https://facebook.com/sharer/sharer.php?u=https://www.usclimatedata.com/climate/washington/district-of-columbia/united-states/usdc0001" rel="noopener noreferrer" target="_blank"><div class="resp-sharing-button resp-sharing-button--facebook resp-sharing-button--small"><div aria-hidden="

In [5]:
print(soup.p.parent.prettify())
# prettify() is handy for formatted printing

<div class="float-left mb-4 mt-2">
 <p class="selection_title">
  Select a state by name
 </p>
</div>



In [6]:
# We then need all the state links on this page
for link in soup.find_all('a'):
    print(link.get('href'))

/
#
/
/climate/united-states/us
/
/climate/united-states/us
/climate/alabama/united-states/3170
/climate/alaska/united-states/3171
/climate/arizona/united-states/3172
/climate/arkansas/united-states/3173
/climate/california/united-states/3174
/climate/colorado/united-states/3175
/climate/connecticut/united-states/3176
/climate/delaware/united-states/3177
/climate/district-of-columbia/united-states/3178
/climate/florida/united-states/3179
/climate/georgia/united-states/3180
/climate/hawaii/united-states/3181
/climate/idaho/united-states/3182
/climate/illinois/united-states/3183
/climate/indiana/united-states/3184
/climate/iowa/united-states/3185
/climate/kansas/united-states/3186
/climate/kentucky/united-states/3187
/climate/louisiana/united-states/3188
/climate/maine/united-states/3189
/climate/maryland/united-states/1872
/climate/massachusetts/united-states/3191
/climate/michigan/united-states/3192
/climate/minnesota/united-states/3193
/climate/mississippi/united-states/3194
/climate/

In [7]:
# We see the result above includes some links we don't want, so we need to filter those out. 
# For example, some of them are hashtags and some of them are links to other sites
# What we want is all these state links
base_url = 'http://www.usclimatedata.com'
state_links = []
for link in soup.find_all('a'):
    url = link.get('href')
    if url and '/climate/' in url and '/climate/united-states/us' not in url:
        state_links.append(url)
print(len(state_links)) 
# in the end, we get 51 state links, including Washington DC.

53


In [8]:
r = requests.get(base_url + state_links[5])
soup = BeautifulSoup(r.text)
print(soup.title.string)
# Here what we do is to test getting the data for one state to see if this works well

Climate Colorado - Temperature, Rainfall and Averages


In [9]:
rows = soup.find_all('tr')
print(len(rows))
# The data we need is in tr tags

12


In [13]:
rows = [row for row in rows if 'Average high' in str(row)]
print(len(rows))

2


In [15]:
high_temps = []
for row in rows:
    tds = row.find_all('td')
    for i in range(1,6):
        high_temps.append(tds[i].text)
print(high_temps)
# filter rows, and add temp data to a list

['46', '54', '61', '72', '82', '88', '79', '66', '52', '45']


In [16]:
r = requests.get(base_url + state_links[5])
soup = BeautifulSoup(r.text)
print(soup.title.string)
# Here we test getting the data for one state again and then print the title for that page

Climate Colorado - Temperature, Rainfall and Averages


In [17]:
state = soup.title.string.split()[1]
print(state)
s = soup.title.string
state = s[s.find(' '):s.find('-')].strip()
print(state)
# get the name of the state

Colorado
Colorado


In [19]:
data = {}
data[state] = high_temps
print(data)
# for a single state, this is what our scraped data looks like

{'Colorado': ['46', '54', '61', '72', '82', '88', '79', '66', '52', '45']}


In [23]:
data = {}
for state_link in state_links:
    url = base_url + state_link
    r = requests.get(base_url + state_link)
    soup = BeautifulSoup(r.text)
    rows = soup.find_all('tr')
    rows = [row for row in rows if 'Average high' in str(row)]
    high_temps = []
    for row in rows:
        tds = row.find_all('td')
        for i in range(1,6):
            high_temps.append(tds[i].text)
    s = soup.title.string
    state = s[s.find(' '):s.find('-')].strip()
    data[state] = high_temps
print(data)
# Here we loop through our 51-state list, and get high temp data for each state

{'Alabama': ['58', '67', '74', '82', '88', '91', '85', '75', '65', '56'], 'Alaska': ['27', '34', '44', '56', '63', '64', '55', '40', '28', '25'], 'Arizona': ['71', '77', '85', '95', '104', '104', '100', '89', '76', '66'], 'Arkansas': ['55', '64', '73', '81', '89', '93', '86', '75', '63', '52'], 'California': ['60', '65', '71', '80', '87', '91', '87', '78', '64', '54'], 'Colorado': ['46', '54', '61', '72', '82', '88', '79', '66', '52', '45'], 'Connecticut': ['40', '47', '58', '68', '77', '81', '74', '63', '53', '42'], 'Delaware': ['47', '55', '66', '75', '83', '85', '79', '69', '58', '47'], 'District Of Columbia': ['44', '53', '64', '75', '83', '84', '78', '67', '55', '45'], 'Florida': ['67', '74', '80', '87', '91', '92', '88', '81', '73', '65'], 'Georgia': ['57', '64', '72', '81', '86', '88', '82', '73', '64', '54'], 'Hawaii': ['80', '81', '83', '85', '87', '89', '89', '87', '84', '81'], 'Idaho': ['45', '55', '62', '72', '81', '90', '79', '65', '48', '38'], 'Illinois': ['36', '46', '59

In [24]:
import csv
# save to CSV file

In [ ]:
with open('high_temps.csv','w') as f:
    w = csv.writer(f)
    w.writerows(data.items())